In [1]:
import os
import random

import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, use_memory_fs=True, nb_workers=12)
import numpy as np

from tqdm.auto import tqdm
tqdm.pandas()

from utils.metrics import f1_macro

data_path = "./data/"
mapping_path = "./mapping/"

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df_train = pd.read_csv(os.path.join(data_path, "addresses-train.csv"), encoding='windows-1251', dtype=str)
df_train = df_train.astype({'source': 'int64'})
df_train.shape

(15337320, 13)

In [3]:
df_train.head()

,address,region,region_type,municipality,municipality_type,settlement,settlement_type,location,location_type,street,street_type,house,source
0,а.обл. Еврейская; ; г Биробиджан; ; переулок...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Авангардный,переулок,дом 1,2
1,А.обл. Еврейская; ; Г Биробиджан; ; пер. Ава...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Авангардный,переулок,дом 3,2
2,Аобл Еврейская; ; город Биробиджан; ; переул...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Авангардный,переулок,дом 4,1
3,аобл Еврейская; ; г. Биробиджан; ; переулок ...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Алмазный,переулок,дом 4,2
4,Аобл Еврейская; ; Г. Биробиджан; ; пер-к Алм...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Алмазный,переулок,дом 5,1


In [4]:
df_pred = pd.DataFrame(df_train["address"].str.split(';').tolist(), columns=["region_str", "municipality_str", "settlement_str", "location_str", "street_str", "house_str", "source_str"])
df_pred.head()

,region_str,municipality_str,settlement_str,location_str,street_str,house_str,source_str
0,а.обл. Еврейская,,г Биробиджан,,переулок Авангардный,дом 1,2
1,А.обл. Еврейская,,Г Биробиджан,,пер. Авангардный,дом 3,2
2,Аобл Еврейская,,город Биробиджан,,переулок Авангардный,дом 4,1
3,аобл Еврейская,,г. Биробиджан,,переулок Алмазный,дом 4,2
4,Аобл Еврейская,,Г. Биробиджан,,пер-к Алмазный,дом 5,1


In [5]:
df_pred = pd.concat([df_train, df_pred], axis=1)

In [6]:
def replace_str(row):
    columns = row.index.tolist()
    columns.sort(key=lambda x: len(x))
    if row[columns[1]] == '  ':
        return '  '
    result = row[columns[1]].replace(row[columns[0]], '')
    result = result.strip(' ')
    return result

In [7]:
df_pred["region_type_str"] = df_pred[["region_str", "region"]].parallel_apply(replace_str, axis=1)
df_pred["municipality_type_str"] = df_pred[["municipality_str", "municipality"]].parallel_apply(replace_str, axis=1)
df_pred["settlement_type_str"] = df_pred[["settlement_str", "settlement"]].parallel_apply(replace_str, axis=1)
df_pred["location_type_str"] = df_pred[["location_str", "location"]].parallel_apply(replace_str, axis=1)
df_pred["street_type_str"] = df_pred[["street_str", "street"]].parallel_apply(replace_str, axis=1)

In [8]:
df_pred.head()

,address,region,region_type,municipality,municipality_type,settlement,settlement_type,location,location_type,street,...,settlement_str,location_str,street_str,house_str,source_str,region_type_str,municipality_type_str,settlement_type_str,location_type_str,street_type_str
0,а.обл. Еврейская; ; г Биробиджан; ; переулок...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Авангардный,...,г Биробиджан,,переулок Авангардный,дом 1,2,а.обл.,,г,,переулок
1,А.обл. Еврейская; ; Г Биробиджан; ; пер. Ава...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Авангардный,...,Г Биробиджан,,пер. Авангардный,дом 3,2,А.обл.,,Г,,пер.
2,Аобл Еврейская; ; город Биробиджан; ; переул...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Авангардный,...,город Биробиджан,,переулок Авангардный,дом 4,1,Аобл,,город,,переулок
3,аобл Еврейская; ; г. Биробиджан; ; переулок ...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Алмазный,...,г. Биробиджан,,переулок Алмазный,дом 4,2,аобл,,г.,,переулок
4,Аобл Еврейская; ; Г. Биробиджан; ; пер-к Алм...,Еврейская,автономная область,NaN,NaN,Биробиджан,город,NaN,NaN,Алмазный,...,Г. Биробиджан,,пер-к Алмазный,дом 5,1,Аобл,,Г.,,пер-к


In [9]:
df_pred.groupby(["region_type", "region_type_str"])["region_type_str"].count().to_csv(os.path.join(mapping_path, "region.csv"))
df_pred.groupby(["municipality_type", "municipality_type_str"])["municipality_type_str"].count().to_csv(os.path.join(mapping_path, "municipality.csv"))
df_pred.groupby(["settlement_type", "settlement_type_str"])["settlement_type_str"].count().to_csv(os.path.join(mapping_path, "settlement.csv"))
df_pred.groupby(["location_type", "location_type_str"])["location_type_str"].count().to_csv(os.path.join(mapping_path, "location.csv"))
df_pred.groupby(["street_type", "street_type_str"])["street_type_str"].count().to_csv(os.path.join(mapping_path, "street.csv"))

In [9]:
# region_mapping = df_pred.groupby("region_type")["region_type_str"].apply(set).apply(list)
# municipality_mapping = df_pred.groupby("municipality_type")["municipality_type_str"].apply(set).apply(list)
# settlement_mapping = df_pred.groupby("settlement_type")["settlement_type_str"].apply(set).apply(list)
# location_mapping = df_pred.groupby("location_type")["location_type_str"].apply(set).apply(list)
# street_mapping = df_pred.groupby("street_type")["street_type_str"].apply(set).apply(list)

In [10]:
# pd.DataFrame((region_mapping.values, region_mapping.index.tolist())).T.rename(columns={0: "value", 1: "key"}).to_csv(os.path.join(mapping_path, "region.csv"), index=False)
# pd.DataFrame((municipality_mapping.values, municipality_mapping.index.tolist())).T.rename(columns={0: "value", 1: "key"}).to_csv(os.path.join(mapping_path, "municipality.csv"), index=False)
# pd.DataFrame((settlement_mapping.values, settlement_mapping.index.tolist())).T.rename(columns={0: "value", 1: "key"}).to_csv(os.path.join(mapping_path, "settlement.csv"), index=False)
# pd.DataFrame((location_mapping.values, location_mapping.index.tolist())).T.rename(columns={0: "value", 1: "key"}).to_csv(os.path.join(mapping_path, "location.csv"), index=False)
# pd.DataFrame((street_mapping.values, street_mapping.index.tolist())).T.rename(columns={0: "value", 1: "key"}).to_csv(os.path.join(mapping_path, "street.csv"), index=False)